<a href="https://colab.research.google.com/github/ferzok/PatternDiscovery/blob/master/Copy_of_5m_12candles_strong_2016_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/5m.Si_160101_180930.csv',delimiter=';')  
dataset=dataset.rename(columns={ dataset.columns[0]: "date" })
dataset['date'] =  pd.to_datetime(dataset['date'], format='%Y%m%d')
dataset=dataset.rename(columns={ dataset.columns[1]: "time" })
dataset=dataset.rename(columns={ dataset.columns[2]: "open" })
dataset=dataset.rename(columns={ dataset.columns[3]: "high" })
dataset=dataset.rename(columns={ dataset.columns[4]: "low" })
dataset=dataset.rename(columns={ dataset.columns[5]: "close" })
dataset=dataset.rename(columns={ dataset.columns[6]: "vol" })
dataset['weekday'] = dataset['date'].dt.dayofweek
# next 12 periods for 100 points
k = 12
points=100
res = dataset.high.rolling(k).max().shift(-k)
dataset['NextHigh'+str(points)]=res
res = dataset.low.rolling(k).min().shift(-k)
dataset['NextLow'+str(points)]=res
dataset['prUp'+str(points)]=dataset.loc[ : , 'NextHigh'+str(points)]-dataset.high-points>0
dataset['prDn'+str(points)]=dataset.low-dataset.loc[ : , 'NextLow'+str(points)]-points>0
# next 96 periods for 300 points
k = 96
points=300
res = dataset.high.rolling(k).max().shift(-k)
dataset['NextHigh'+str(points)]=res
res = dataset.low.rolling(k).min().shift(-k)
dataset['NextLow'+str(points)]=res
dataset['prUp'+str(points)]=(dataset.loc[ : , 'NextHigh'+str(points)]-dataset.high-points>0) & (dataset.low-dataset.loc[ : , 'NextLow'+str(points)]-points<0)
dataset['prDn'+str(points)]=(dataset.loc[ : , 'NextHigh'+str(points)]-dataset.high-points<0) & (dataset.low-dataset.loc[ : , 'NextLow'+str(points)]-points>0)
dataset['date'] = dataset['date'].dt.dayofweek# end
# dataset = dataset.drop(dataset[dataset.time > 230000].index)
t=12
nrows=dataset.shape[0]
df=pd.DataFrame({'time': dataset.iloc[t-1:nrows,1]})
df['weekday']=dataset.iloc[t-1:nrows,7].values
for i in range(0, t):
   df['HL'+str(i)]=dataset.iloc[t-1-i:nrows-i,3].values-dataset.iloc[t-1-i:nrows-i,4].values
   df['HL'+str(i)]=sc.fit_transform(df['HL'+str(i)])
   df['HO'+str(i)]=dataset.iloc[t-1-i:nrows-i,3].values-dataset.iloc[t-1-i:nrows-i,2].values
   df['HC'+str(i)]=dataset.iloc[t-1-i:nrows-i,3].values-dataset.iloc[t-1-i:nrows-i,5].values
   df['V'+str(i)]=dataset.iloc[t-1-i:nrows-i,6].values
   df['C'+str(i)]=dataset.iloc[t-1-i:nrows-i,5].values
df['pr100Up']=dataset.iloc[t-1:nrows,10].values
df['pr100Dn']=dataset.iloc[t-1:nrows,11].values
df['pr300Up']=dataset.iloc[t-1:nrows,14].values
df['pr300Dn']=dataset.iloc[t-1:nrows,15].values
df = df.drop(df[df.time > 230000].index)
df = df.drop(df[df.time < 110000].index)
df = df.reset_index(drop=True)

numberinputs=df.shape[1]-4
X = df.iloc[:, 0:numberinputs].values
y_UP100 = df.iloc[:, numberinputs].values
y_DOWN100 = df.iloc[:, numberinputs+1].values
y_UP300 = df.iloc[:, numberinputs+2].values
y_DOWN300 = df.iloc[:, numberinputs+3].values

# Encoding categorical data

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train_DOWN100, y_test_DOWN100 = train_test_split(X, y_DOWN100, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_UP100, y_test_UP100 = train_test_split(X, y_UP100, test_size = 0.2, random_state = 0)

X_train, X_test, y_train_UP300, y_test_UP300 = train_test_split(X, y_UP300, test_size = 0.2, random_state = 0)
X_train, X_test, y_train_DOWN300, y_test_DOWN300 = train_test_split(X, y_DOWN300, test_size = 0.2, random_state = 0)


# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
classifierUP100 = Sequential()
classifierDN100 = Sequential()
classifierUP300 = Sequential()
classifierDN300 = Sequential()

# Adding the input layer and the first hidden layer
classifierUP100.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = numberinputs))
classifierDN100.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = numberinputs))
classifierUP300.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = numberinputs))
classifierDN300.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = numberinputs))


# Adding the second hidden layer
classifierUP100.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))


classifierDN100.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))

classifierUP300.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))


classifierDN300.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifierUP100.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifierDN100.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifierUP300.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifierDN300.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))


# Compiling the ANN
classifierUP100.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifierDN100.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifierUP300.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifierDN300.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



# Fitting the ANN to the Training set
classifierUP100.fit(X_train, y_train_UP100, batch_size = 128, epochs =50)
classifierDN100.fit(X_train, y_train_DOWN100, batch_size = 1024, epochs = 50)
classifierUP300.fit(X_train, y_train_UP300, batch_size = 1024, epochs = 50)
classifierDN300.fit(X_train, y_train_DOWN300, batch_size = 1024, epochs = 50)

from google.colab import files


model_jsonup100 = classifierUP100.to_json()
with open('classifierUP100.json', "w") as json_file:
    json_file.write(model_jsonup100)
json_file.close
files.download('classifierUP100.json')
# serialize weights to HDF5
classifierUP100.save_weights('classifierUP100.h5')
files.download('classifierUP100.h5')


model_json_dn100 = classifierDN100.to_json()
with open('classifierDN100.json', "w") as json_file:
    json_file.write(model_json_dn100)
json_file.close
files.download('classifierDN100.json')
# serialize weights to HDF5
classifierDN100.save_weights('classifierDN100.h5')
files.download('classifierDN100.h5')


model_jsonUP300 = classifierUP300.to_json()
with open('classifierUP300.json', "w") as json_file:
    json_file.write(model_jsonUP300)
json_file.close
files.download('classifierUP300.json')
# serialize weights to HDF5
classifierUP300.save_weights('classifierUP300.h5')
files.download('classifierUP300.h5')


model_jsonDN300 = classifierDN300.to_json()
with open('classifierDN300.json', "w") as json_file:
    json_file.write(model_jsonDN300)
files.download('classifierDN300.json')
# serialize weights to HDF5
classifierDN300.save_weights('classifierDN300.h5')
files.download('classifierDN300.h5')
print("Saved model to drive")

# Predicting the Test set results
from sklearn.metrics import confusion_matrix
y_pred_DOWN100 = classifierDN100.predict(X_test)
y_pred_DOWN100 = (y_pred_DOWN100 >0.7)
cm_DOWN100 = confusion_matrix(y_test_DOWN100, y_pred_DOWN100)
print(cm_DOWN100)

y_pred_UP100 = classifierUP100.predict(X_test)
y_pred_UP100 = (y_pred_UP100 >0.8)
cm_UP100 = confusion_matrix(y_test_UP100, y_pred_UP100)
print(cm_UP100)

y_pred_DOWN300 = classifierDN300.predict(X_test)
y_pred_DOWN300 = (y_pred_DOWN300> 0.7)
cm_DOWN300 = confusion_matrix(y_test_DOWN300, y_pred_DOWN300)

y_pred_UP300 = classifierUP300.predict(X_test)
y_pred_UP300 = (y_pred_UP300 >0.7)
cm_UP300 = confusion_matrix(y_test_UP300, y_pred_UP300)
nrows=y_test_UP100.shape[0]
allresults=pd.DataFrame({'time': df.iloc[0:nrows,0]})
#allresults['date']=df.iloc[0:nrows,6] 
allresults['Price']=df.iloc[0:nrows,6] 
allresults['UP100']=y_pred_UP100 
allresults['UP100Real']=y_test_UP100 
allresults['DN100']=y_pred_DOWN100 
allresults['DN100Real']=y_test_DOWN100 
allresults['UP300']=y_pred_UP300 
allresults['UP300Real']=y_test_UP300 
allresults['DN300']=y_pred_DOWN300 
allresults['DN300Real']=y_test_DOWN300 
from google.colab import files
allresults.to_csv('full_results.csv', sep=';')
files.download('full_results.csv')


#CHECK current results
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SPFB.Si_181112_181112.csv',delimiter=';')  
dataset=dataset.rename(columns={ dataset.columns[0]: "date" })
dataset['date'] =  pd.to_datetime(dataset['date'], format='%Y%m%d')
dataset=dataset.rename(columns={ dataset.columns[1]: "time" })
dataset=dataset.rename(columns={ dataset.columns[2]: "open" })
dataset=dataset.rename(columns={ dataset.columns[3]: "high" })
dataset=dataset.rename(columns={ dataset.columns[4]: "low" })
dataset=dataset.rename(columns={ dataset.columns[5]: "close" })
dataset=dataset.rename(columns={ dataset.columns[6]: "vol" })
dataset['weekday'] = dataset['date'].dt.dayofweek
# next 24 periods for 100 points
k = 12
points=100
res = dataset.high.rolling(k).max().shift(-k)
dataset['NextHigh'+str(points)]=res
res = dataset.low.rolling(k).min().shift(-k)
dataset['NextLow'+str(points)]=res
dataset['prUp'+str(points)]=dataset.loc[ : , 'NextHigh'+str(points)]-dataset.close-points>0
dataset['prDn'+str(points)]=dataset.close-dataset.loc[ : , 'NextLow'+str(points)]-points>0
# next 96 periods for 300 points
k = 96
points=300
res = dataset.high.rolling(k).max().shift(-k)
dataset['NextHigh'+str(points)]=res
res = dataset.low.rolling(k).min().shift(-k)
dataset['NextLow'+str(points)]=res
dataset['prUp'+str(points)]=dataset.loc[ : , 'NextHigh'+str(points)]-dataset.close-points>0
dataset['prDn'+str(points)]=dataset.close-dataset.loc[ : , 'NextLow'+str(points)]-points>0
# end
dataset = dataset.drop(dataset[dataset.time > 230000].index)
t=12
nrows=dataset.shape[0]
dfx=pd.DataFrame({'time': dataset.iloc[t-1:nrows,1]})
dfx['weekday']=dataset.iloc[t-1:nrows,7].values
for i in range(0, t):
   dfx['HL'+str(i)]=dataset.iloc[t-1-i:nrows-i,3].values-dataset.iloc[t-1-i:nrows-i,4].values
   dfx['HO'+str(i)]=dataset.iloc[t-1-i:nrows-i,3].values-dataset.iloc[t-1-i:nrows-i,2].values
   dfx['HC'+str(i)]=dataset.iloc[t-1-i:nrows-i,3].values-dataset.iloc[t-1-i:nrows-i,5].values
   dfx['V'+str(i)]=dataset.iloc[t-1-i:nrows-i,6].values
   dfx['С'+str(i)]=dataset.iloc[t-1-i:nrows-i,5].values
dfx['pr100Up']=dataset.iloc[t-1:nrows,10].values
dfx['pr100Dn']=dataset.iloc[t-1:nrows,11].values
dfx['pr300Up']=dataset.iloc[t-1:nrows,14].values
dfx['pr300Dn']=dataset.iloc[t-1:nrows,15].values
dfx = dfx.reset_index(drop=True)
dfxrows=dfx.shape[0]
Xx = dfx.iloc[:, 0:numberinputs].values  
sc = StandardScaler()
Xx = sc.fit_transform(Xx)
y_pred_UP100 = classifierUP100.predict(Xx) 
y_pred_DOWN100 = classifierDN100.predict(Xx)
y_pred_UP300 = classifierUP300.predict(Xx) 
y_pred_DOWN300 = classifierDN300.predict(Xx)

allresults=pd.DataFrame({'time': dfx.iloc[0:nrows,0]})
allresults['Price']=dfx.iloc[0:nrows,6] 
allresults['UP100']=y_pred_UP100 
allresults['DN100']=y_pred_DOWN100 
allresults['UP300']=y_pred_UP300 
allresults['DN300']=y_pred_DOWN300 

from google.colab import files
allresults.to_csv('60min_results.csv', sep=';')
files.download('60min_results.csv')


11         97.0
12        170.0
13        131.0
14        154.0
15        187.0
16        150.0
17        150.0
18        111.0
19        154.0
20         88.0
21        110.0
22        202.0
23         54.0
24         60.0
25        115.0
26         73.0
27        163.0
28         81.0
29        148.0
30        155.0
31        151.0
32        115.0
33         56.0
34        108.0
35         64.0
36         63.0
37        146.0
38         69.0
39        122.0
40        163.0
          ...  
111868     17.0
111869     17.0
111870     22.0
111871     20.0
111872     26.0
111873     48.0
111874     22.0
111875     16.0
111876     20.0
111877     40.0
111878     18.0
111879     18.0
111880     28.0
111881     21.0
111882     62.0
111883     30.0
111884     73.0
111885     25.0
111886     20.0
111887     45.0
111888     36.0
111889     39.0
111890     31.0
111891      9.0
111892     52.0
111893     67.0
111894     34.0
111895     21.0
111896     25.0
111897     37.0
Name: HL0, Length: 11188